In [ ]:
# import os
# ! git clone https://github.com/olafbar/classifiers.git
# ! pip install mahotas

In [ ]:
#########################################################################  
classifier='etc'  #nazwa klasyfikatora
## etc,bagsvc,gbc,rf,vot,mlp,nsv,svm,knn,ovo_mlp, ovo_svm, ovr_mlp
dir='42'  # nazwa katalogu z obrazkami ANDROID (2 cyfry)
# url dla ANDROID
url='https://user.credo.science/user-interface/download/images/download.php?name='+dir+'.zip -O '+dir+'.zip'
#########################################################################
# dir='001'  # nazwa katalogu z obrazkami dla IOS (3 cyfry)
# #url dla IOS
# url='https://user.credo.science/user-interface/download/images_ios/download.php?name='+dir+'.zip -O '+dir+'.zip'

print(url)
os.system('wget %s' %url)
par=dir+' -d '+dir
os.system('unzip %s' %par)



https://user.credo.science/user-interface/download/images/download.php?name=42.zip -O 42.zip


0

In [ ]:
import cv2
import glob

images=[]
list_images_name=[]
for img in glob.glob(dir+"/*.png"):
    n = cv2.imread(img)
    img = img.split('/')
    list_images_name.append(img[1])
    images.append(n)

In [ ]:
import mahotas
import numpy as np
rows , cols, _ =images[0].shape
radius=np.sqrt((rows/2)**2+(cols/2.)**2)
features2 =[]
for img in images:
  blackwhite=img[:,:,0]+img[:,:,1]+img[:,:,2]
  threshold = blackwhite.mean() + blackwhite.std() * 5
  threshold = threshold if threshold < 100 else 100  
  mask = np.where(blackwhite > threshold, 1, 0)
  blackwhite = blackwhite * mask
  zm = mahotas.features.zernike_moments(blackwhite, radius, degree=8, cm=(rows/2., cols/2.))
  zm = np.hstack([zm, blackwhite.mean()])
  features2.append(zm)
feature_array2=np.array(features2)

In [ ]:
import pickle

from joblib import dump, load
pkl_filename='classifiers/'+classifier+'.pkl'
scaler_filename='classifiers/'+classifier+'_scaler.pkl'
# print(scaler_filename)
clf1=load(pkl_filename)
scaler2=load(scaler_filename)

In [ ]:
X_std2 = scaler2.transform(feature_array2)
y_pred2 = clf1.predict(X_std2)

In [ ]:
for i in range(4):
    count = sum(map(lambda x : x==i, y_pred2))
    print(i,count)

0 5141
1 7804
2 16694
3 10361


In [ ]:
classifiers=[classifier for x in range(len(y_pred2))]
import pandas as pd
df = pd.DataFrame({
    'Classifier': classifiers,
    'Hit ID': list_images_name,
    'Class': y_pred2
})

In [ ]:
df.to_csv(classifier+'_'+dir+'.csv', index=False)

In [ ]:
print(df.loc[df['Class'] == 2])

In [ ]:
list_images_name[39998]

In [ ]:
from matplotlib import pyplot as plt
plt.figure()
plt.imshow(images[39998])